In [ ]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
sys.path.insert(0, '/src')
from eliot import to_file
to_file(sys.stdout)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

processed_path = Path('/data/processed')
raw_path = Path('/data/raw')

In [ ]:
base_df = pd.read_parquet(processed_path/'final_df.parquet')

### Remove features which have 98% 0 values

In [ ]:
cols = []
def na_analysis(df):
    lst = []
    total_rows = df.shape[0]
    for col in df.columns:
        # Sum of NaN values in a column
        na_values = df[col].eq(0).sum()
        lst.extend([[col,total_rows,na_values,(na_values/total_rows)*100]])
        if ((na_values/total_rows)*100) == 100 and (col not in cols):
            cols.append(col)

    return lst


df_na = pd.DataFrame(
    na_analysis(base_df),
    columns=['column_name','total_count','null_values','%_null_values']
)

df_na.sort_values(['%_null_values'],ascending=False,inplace=True)

print(f'Total values above cutoff = {len(cols)}')
df_na.head(10)

In [ ]:
print(base_df.shape)

In [ ]:
drop_cols = df_na[
    (df_na['%_null_values'] == 100) & (~df_na['column_name'].str.startswith('hosp_target'))
]['column_name']
base_df.drop(drop_cols,
        axis=1,
        inplace=True
       )

In [ ]:
drop_cols

In [ ]:
print(base_df.shape)

In [ ]:
base_df.to_parquet(processed_path/'final_cleaned_df.parquet')

## =======================END=====================

In [ ]:
base_df = pd.read_parquet(processed_path/'05-result.parquet')


In [ ]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
import pickle

### Pre-process the dataframe before running feature selection

In [ ]:
output_cols = [col for col in base_df.columns if 'hosp_target' in col]
x_df = base_df[base_df.columns.difference(output_cols)]
y_df = base_df[output_cols]
print(x_df.shape)
print(y_df.shape)

In [ ]:
exclude_cols = ['masterpatientid','censusdate', 'facilityid', 'bedid', 'client']

x_df = x_df[x_df.columns.difference(exclude_cols)]
x_df.shape

y_df = y_df.fillna(False)
y_df['hosp_target_3_day_hosp'] = y_df['hosp_target_3_day_hosp'].astype('float32')
target_3_day = y_df['hosp_target_3_day_hosp']

In [ ]:
def fill_na_train(df):
    # Fill Median value for all NaN's in the respective columns
    has_na = df.isna().sum() > 0
    d = df.loc[:, has_na].median()
    df = df.fillna(d)
    
    return df, d

def fill_na_valid_or_test(df, na_filler):
    return df.fillna(na_filler)


x_df, na_filler = fill_na_train(x_df)
x_df = x_df.astype('float32')

In [ ]:
print(x_df.shape)
print(y_df.shape)
print(len(target_3_day))

In [ ]:
x_df.to_parquet(processed_path/'x_df.parquet')
with open(processed_path/'target_3_day.pickle','wb') as f: pickle.dump(target_3_day, f, protocol=4)

In [ ]:
x_df = pd.read_parquet(processed_path/'x_df.parquet')
with open(processed_path/'target_3_day.pickle','rb') as f: target_3_day = pickle.load(f)

## Feature Selection 

In [ ]:
%%time

# Correlation for all features with the target

corr_matrix = x_df.corrwith(y_df['hosp_target_3_day_hosp'])

_df = pd.DataFrame({'cols':corr_matrix.index, 'value':corr_matrix.values})
_df.sort_values(by='value',ascending=False).head(2000)

In [ ]:
%%time
## Remove constant features

constant_features = []
for feat in x_df.columns:
    # convert all features to Float32
    
    if x_df[feat].std() == 0:
        constant_features.append(feat)

print(constant_features)

# x_df.drop(labels=constant_features, axis=1, inplace=True)


In [ ]:
%%time

# Remove duplicated features

duplicated_features = []
for i in range(0, len(x_df.columns)):
    col_1 = x_df.columns[i]

    for col_2 in x_df.columns[i + 1:]:
        if x_df[col_1].equals(x_df[col_2]):
            duplicated_features.append(col_2)

print(duplicated_features)

# x_df.drop(labels=duplicated_features, axis=1, inplace=True)

In [ ]:
%%time


# calculate the mutual information between the variables and the target
# this returns the mutual information value of each feature.
# the smaller the value the less information the feature has about the target


mi = mutual_info_classif(x_df.fillna(0), target_3_day)
print(mi)

# let's add the variable names and order the features
# according to the MI for clearer visualisation
mi = pd.Series(mi)
mi.index = x_df.columns
mi = mi.sort_values(ascending=False)
mi.to_csv('mi-date_cols.csv', header=True)
# and now let's plot the ordered MI values per feature
mi.sort_values(ascending=False).plot.bar(figsize=(20, 8))

In [ ]:
%%time

# here I will select the top 10 features
# which are shown below
sel_ = SelectKBest(mutual_info_classif, k=10).fit(x_df.fillna(0), target_3_day)
x_df.columns[sel_.get_support()]

In [ ]:
%%time

# calculate the chi2 p_value between each of the variables
# and the target
# it returns 2 arrays, one contains the F-Scores which are then
# evaluated against the chi2 distribution to obtain the pvalue
# the pvalues are in the second array, see below

f_score = chi2(x_df, target_3_day)
f_score

Keep in mind, that contrarily to MI, where we were interested in the higher MI values,
for Fisher score, the smaller the p_value, the more significant the feature is to predict the target.

One thing to keep in mind when using Fisher score or univariate selection methods,
is that in very big datasets, most of the features will show a small p_value,
and therefore look like they are highly predictive.
This is in fact an effect of the sample size. So care should be taken when selecting features
using these procedures. An ultra tiny p_value does not highlight an ultra-important feature,
it rather indicates that the dataset contains too many samples.

If the dataset contained several categorical variables, we could then combine this procedure with
SelectKBest or SelectPercentile, as I did in the previous lecture.

In [ ]:
%%time

# let's add the variable names and order it for clearer visualisation

pvalues = pd.Series(f_score[1])
pvalues.index = x_df.columns
pvalues.sort_values(ascending=True)


In [ ]:
%%time

# LASSO Regularization

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

logistic = LogisticRegression(C=1, penalty='l1',solver='liblinear',random_state=7).fit(x_df,target_3_day)
model = SelectFromModel(logistic, prefit=True)

# x_new_df = model.transform(x_df)

# this command let's me visualise those features that were kept
model.get_support()

In [ ]:
%%time

# Now I make a list with the selected features
selected_feat = x_df.columns[(model.get_support())]

print('total features: {}'.format((x_df.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(model.estimator_.coef_ == 0)))



In [ ]:
%%time

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# step forward feature selection
# I indicate that I want to select 10 features from
# the total, and that I want to select those features
# based on the optimal roc_auc

sfs1 = SFS(RandomForestRegressor(),
           k_features=20,
           forward=True,
           floating=False,
           verbose=2,
           scoring='r2',
           cv=3)

sfs1 = sfs1.fit(np.array(x_df), target_3_day)
selected_feat= x_df.columns[list(sfs1.k_feature_idx_)]
selected_feat

In [ ]:
!pip install mlxtend